In [1]:
import sys
if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")

import torch
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import DataLoader
from tqdm import tqdm

from data_utility import *
from data_utils import *
from loss import *
from train import *
from deeplab_model.deeplab import *
from dense_vnet.DenseVNet import DenseVNet
from sync_batchnorm import convert_model
import datetime

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
USE_GPU = True
NUM_WORKERS = 12
BATCH_SIZE = 1

dtype = torch.float32 
# define dtype, float is space efficient than double

if USE_GPU and torch.cuda.is_available():
    
    device = torch.device("cuda:0")
    
    torch.backends.cudnn.benchmark = True
    torch.backends.cudnn.enabled = True
    # magic flag that accelerate
    
    print('using GPU for training')
else:
    device = torch.device('cpu')
    print('using CPU for training')

using GPU for training


In [3]:
train_dataset = get_full_resolution_dataset(data_type = 'nii_train', 
                transform=transforms.Compose([
                random_affine(90, 15),
                random_filp(0.5)]))
# do data augumentation on train dataset

validation_dataset = get_full_resolution_dataset(data_type = 'nii_test', 
                transform=None)
# no data augumentation on validation dataset

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True,
                    num_workers=NUM_WORKERS)
validation_loader = DataLoader(validation_dataset, batch_size=BATCH_SIZE, shuffle=True,
                    num_workers=NUM_WORKERS) # drop_last
# loaders come with auto batch division and multi-thread acceleration

In [4]:
from bv_refinement_network.RefinementModel import RefinementModel, RefinementModel_NoDown

if torch.cuda.device_count() > 1:
    print("Let's use", torch.cuda.device_count(), "GPUs!")
    # dim = 0 [30, xxx] -> [10, ...], [10, ...], [10, ...] on 3 GPUs
    
refine_model = RefinementModel_NoDown(num_classes=1)
refine_model = nn.DataParallel(refine_model)
refine_model = convert_model(refine_model)
refine_model = refine_model.to(device, dtype)

optimizer = optim.Adam(refine_model.parameters(), lr=1e-2)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer)

deeplab = DeepLab(output_stride=16)
deeplab = nn.DataParallel(deeplab)
deeplab = convert_model(deeplab)

checkpoint = torch.load('../deeplab_dilated_save/2019-08-10 09:28:43.844872 epoch: 1160.pth') # best one

deeplab.load_state_dict(checkpoint['state_dict_1'])
deeplab = deeplab.to(device, dtype)

epoch = 0

In [ ]:
'''
test_dictionary = train_dataset[33]

image_1 = test_dictionary['image1_data'].view(1, 1, 256, 256, 256)
label_1 = test_dictionary['image1_label'].view(1, 3, 256, 256, 256)
bv_label = label_1.narrow(1,2,1).to(device, dtype)
if get_dimensions(bv_label) == 4:
    bv_label.unsqueeze_(0)

image_1 = image_1.to(device=device, dtype=dtype) 
label_1 = label_1.to(device=device, dtype=dtype)

deeplab.eval()

out_coarse = deeplab(image_1)
bv_coarse = out_coarse.narrow(1,2,1)

if get_dimensions(bv_coarse) == 4:
    bv_coarse.unsqueeze_(0)

seg_image_concat = torch.cat([bv_coarse, image_1], dim=1)
print(bv_label.shape)
print(bv_coarse.shape)
print(dice_loss(bv_coarse, bv_label))
print(seg_image_concat.shape)
'''

"\ntest_dictionary = train_dataset[33]\n\nimage_1 = test_dictionary['image1_data'].view(1, 1, 256, 256, 256)\nlabel_1 = test_dictionary['image1_label'].view(1, 3, 256, 256, 256)\nbv_label = label_1.narrow(1,2,1).to(device, dtype)\nif get_dimensions(bv_label) == 4:\n    bv_label.unsqueeze_(0)\n\nimage_1 = image_1.to(device=device, dtype=dtype) \nlabel_1 = label_1.to(device=device, dtype=dtype)\n\ndeeplab.eval()\n\nout_coarse = deeplab(image_1)\nbv_coarse = out_coarse.narrow(1,2,1)\n\nif get_dimensions(bv_coarse) == 4:\n    bv_coarse.unsqueeze_(0)\n\nseg_image_concat = torch.cat([bv_coarse, image_1], dim=1)\nprint(bv_label.shape)\nprint(bv_coarse.shape)\nprint(dice_loss(bv_coarse, bv_label))\nprint(seg_image_concat.shape)\n"

In [ ]:
epochs = 5000

record = open('train_bv_refine_nodown_resize.txt','a+')

logger = {'train':[], 'validation_1': []}

min_val = 1

for e in tqdm(range(epoch + 1, epochs)):
# iter over epoches
    epoch_loss = 0
        
    for t, batch in enumerate(train_loader):
    # iter over the train mini batches
        #print(batch.shape)
        training_losses = []
        for minibatch in range(BATCH_SIZE):
            refine_model.train()
            deeplab.eval()
            # Set the model flag to train
            # 1. enable dropout
            # 2. batchnorm behave differently in train and test
            #print(batch['image1_data'])
            image_1 = batch['image1_data'][minibatch].to(device=device, dtype=dtype)
            image_1 = image_1.view(1,1,256,256,256)
            
            label_1 = batch['image1_label'][minibatch].to(device=device, dtype=dtype)
            label_1 = label_1.view(1,3,256,256,256)

            bv_label = label_1[:, 2, :, :, :]
            bv_label = bv_label.view(1,1,256,256,256)

            original_res = [a[minibatch].item() for a in batch['original_resolution']]
    
            # Get coarse output from deeplab model from 256 resolution input
            out_coarse = deeplab(image_1)
            out_coarse = out_coarse.view(1,3,256,256,256)

            bv_coarse = out_coarse[:, 2, :, :, :]
            bv_coarse = bv_coarse.view(1,1,256,256,256)
            bv_coarse_binarized = binarize_output(bv_coarse)
            
            bv_x, bv_y, bv_z = loadbvcenter(bv_label.view(1, 256, 256, 256))
            bbox_bv_label = bv_label.view(256, 256, 256)[bv_x-64:bv_x+64, bv_y-64:bv_y+64, bv_z-64:bv_z+64]
            bbox_bv_label = reshape_image(bbox_bv_label.squeeze(), 128, 128, 128).to(device, dtype)
            bbox_bv_label = bbox_bv_label.view(1,1,128,128,128)
            
            pred_x, pred_y, pred_z = loadbvcenter(bv_coarse_binarized.view(1,256,256,256))
            bbox_bv = bv_coarse.view(256, 256, 256)[pred_x-64:pred_x+64, pred_y-64:pred_y+64, pred_z-64:pred_z+64]
            bbox_bv = reshape_image(bbox_bv.squeeze(), 128, 128, 128).to(device, dtype)
            bbox_bv = bbox_bv.view(1,1,128,128,128)

            #bbox_image = get_bounding_box_image(image_1, (256,256,256)).to(device, dtype)
            bbox_image = image_1[:, :, pred_x-64:pred_x+64, pred_y-64:pred_y+64, pred_z-64:pred_z+64]
            bbox_image = reshape_image(bbox_image.squeeze(), 128, 128, 128).to(device, dtype)
            bbox_image = bbox_image.view(1, 1, 128, 128, 128)
            #print(bbox_image.shape)

            #show_image_slice(image_1)
            #show_image_slice(image_1_resize)
            #show_image_slice(label_1)
            #show_image_slice(bv_label)
            #show_image_slice(bv_label_resize)
            #show_image_slice(out_coarse)
            #show_image_slice(bv_coarse)
            #show_image_slice(bv_coarse_resize)
            #show_image_slice(bbox_bv_label)
            #show_image_slice(bbox_bv)
            #show_image_slice(bbox_image)

            bbox_concat = torch.cat([bbox_bv, bbox_image], dim=1)

            refine_out = refine_model(bbox_concat)
            loss = dice_loss(refine_out, bbox_bv_label)
            training_losses.append(loss)
            
            torch.cuda.empty_cache()
            
        # calculate loss
        avg_loss = sum(training_losses) / BATCH_SIZE
        training_losses = []
        
        print(avg_loss)
        epoch_loss += avg_loss.item()
        # record minibatch loss to epoch loss
        
        optimizer.zero_grad()
        # set the model parameter gradient to zero
        
        avg_loss.backward()
        # calculate the gradient wrt loss
        optimizer.step()
        # take a gradient descent step
        
    outstr = 'Epoch {0} finished ! Training Loss: {1:.4f}'.format(e, epoch_loss/(t+1)) + '\n'
    
    logger['train'].append(epoch_loss/(t+1))
    
    print(outstr)
    record.write(outstr)
    record.flush()

    if (e <= 100 and e%5 == 0) or (e > 100 and e%2 == 0):
    # do validation every 5 epoches
        deeplab.eval()
        refine_model.eval()
        # set model flag to eval
        # 1. disable dropout
        # 2. batchnorm behave differs

        with torch.no_grad():
        # stop taking gradient
        
            #valloss_4 = 0
            #valloss_2 = 0
            valloss_1 = 0
            
            for v, vbatch in enumerate(validation_loader):
            # iter over validation mini batches
                val_losses = []
                for minibatch in range(BATCH_SIZE):
                    image_1 = vbatch['image1_data'][minibatch].to(device=device, dtype=dtype)
                    image_1 = image_1.view(1,1,256,256,256)

                    label_1 = vbatch['image1_label'][minibatch].to(device=device, dtype=dtype)
                    label_1 = label_1.view(1,3,256,256,256)

                    bv_label = label_1[:, 2, :, :, :]
                    bv_label = bv_label.view(1,1,256,256,256)

                    original_res = [a[minibatch].item() for a in vbatch['original_resolution']]

                    # Get coarse output from deeplab model from 256 resolution input
                    out_coarse = deeplab(image_1)
                    out_coarse = out_coarse.view(1,3,256,256,256)

                    bv_coarse = out_coarse[:, 2, :, :, :]
                    bv_coarse = bv_coarse.view(1,1,256,256,256)
                    bv_coarse_binarized = binarize_output(bv_coarse)

                    bv_x, bv_y, bv_z = loadbvcenter(bv_label.view(1, 256, 256, 256))
                    bbox_bv_label = bv_label.view(256, 256, 256)[bv_x-64:bv_x+64, bv_y-64:bv_y+64, bv_z-64:bv_z+64]
                    bbox_bv_label = reshape_image(bbox_bv_label.squeeze(), 128, 128, 128).to(device, dtype)
                    bbox_bv_label = bbox_bv_label.view(1,1,128,128,128)
                    
                    pred_x, pred_y, pred_z = loadbvcenter(bv_coarse_binarized.view(1,256,256,256))
                    bbox_bv = bv_coarse.view(256, 256, 256)[pred_x-64:pred_x+64, pred_y-64:pred_y+64, pred_z-64:pred_z+64]
                    #print(bbox_bv.shape)
                    bbox_bv = reshape_image(bbox_bv.squeeze(), 128, 128, 128).to(device, dtype)
                    bbox_bv = bbox_bv.view(1,1,128,128,128)

                    #bbox_image = get_bounding_box_image(image_1, (256,256,256)).to(device, dtype)
                    bbox_image = image_1[:, :, pred_x-64:pred_x+64, pred_y-64:pred_y+64, pred_z-64:pred_z+64]
                    bbox_image = reshape_image(bbox_image.squeeze(), 128, 128, 128).to(device, dtype)
                    bbox_image = bbox_image.view(1, 1, 128, 128, 128)
                    #print(bbox_image.shape)

                    #show_image_slice(image_1)
                    #show_image_slice(image_1_resize)
                    #show_image_slice(label_1)
                    #show_image_slice(bv_label)
                    #show_image_slice(bv_label_resize)
                    #show_image_slice(out_coarse)
                    #show_image_slice(bv_coarse)
                    #show_image_slice(bv_coarse_resize)
                    #show_image_slice(bbox_bv_label)
                    #show_image_slice(bbox_bv)
                    #show_image_slice(bbox_image)

                    bbox_concat = torch.cat([bbox_bv, bbox_image], dim=1)

                    refine_out = refine_model(bbox_concat)
                        
                    loss = dice_loss(refine_out, bbox_bv_label)
                    val_losses.append(loss)
                
                avg_loss = sum(val_losses) / BATCH_SIZE
                val_losses = []
                print(avg_loss)
            
                # calculate loss
                valloss_1 += avg_loss.item()
                torch.cuda.empty_cache()
            
            avg_val_loss = (valloss_1 / (v+1))
            outstr = '------- 1st valloss={0:.4f}'\
                .format(avg_val_loss) + '\n'
            
            logger['validation_1'].append(avg_val_loss)
            #scheduler.step(avg_val_loss)
            
            if avg_val_loss < min_val:
                save_1('refine_bv_nodown_resize_save', refine_model, optimizer, logger, e, scheduler)
            elif e % 10 == 0:
                save_1('refine_bv2_nodown_resize_save', refine_model, optimizer, logger, e, scheduler)
            
            print(outstr)
            record.write(outstr)
            record.flush()
    


record.close()

  0%|          | 0/4999 [00:00<?, ?it/s]

tensor(0.9715, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.9660, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.8324, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.9037, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.8356, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.8612, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.8152, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.9538, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.8322, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.7555, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.7664, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.6831, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.8186, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.7641, device='cuda:0', grad_fn=<DivBackward0>)
tensor(1., device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.7844, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.7061, device='cuda:0', grad_fn=<DivBackward0>)
tensor(1., device='cuda:0', grad_fn=<DivBackward0>)


  0%|          | 1/4999 [05:31<460:03:49, 331.38s/it]

Epoch 1 finished ! Training Loss: 0.8185

tensor(0.6249, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.6841, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.8223, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.7385, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.8501, device='cuda:0', grad_fn=<DivBackward0>)
tensor(1., device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.8137, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.7100, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.7892, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.7846, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.6606, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.6718, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.8127, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.7502, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.6670, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.8400, device='cuda:0', grad_fn=<DivBackward0>)
tensor(1., device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.7350,

  0%|          | 2/4999 [10:56<457:33:25, 329.64s/it]

...

  3%|▎         | 153/4999 [14:44:31<478:31:03, 355.48s/it]

Epoch 153 finished ! Training Loss: 0.7310

tensor(0.7861, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.7342, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.7441, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.7504, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.6794, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.6069, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.7533, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.6679, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.7167, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.6939, device='cuda:0', grad_fn=<DivBackward0>)
tensor(1., device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.8084, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.7021, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.6763, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.7265, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.7649, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.6691, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0

tensor(0.8255, device='cuda:0')
tensor(0.6969, device='cuda:0')
tensor(0.8824, device='cuda:0')
tensor(0.7046, device='cuda:0')
tensor(0.8021, device='cuda:0')
tensor(0.5992, device='cuda:0')
tensor(0.7414, device='cuda:0')
tensor(0.8907, device='cuda:0')
tensor(0.7858, device='cuda:0')
tensor(0.8439, device='cuda:0')
tensor(0.6797, device='cuda:0')
tensor(0.7076, device='cuda:0')
tensor(0.8167, device='cuda:0')
tensor(0.7624, device='cuda:0')
tensor(0.8288, device='cuda:0')
tensor(0.8016, device='cuda:0')
tensor(0.7898, device='cuda:0')
tensor(0.6613, device='cuda:0')
tensor(1., device='cuda:0')
tensor(0.8826, device='cuda:0')
tensor(0.6756, device='cuda:0')
tensor(0.7595, device='cuda:0')
tensor(0.8134, device='cuda:0')
tensor(0.7795, device='cuda:0')
tensor(0.7710, device='cuda:0')
tensor(0.8828, device='cuda:0')
tensor(0.6374, device='cuda:0')
tensor(0.8278, device='cuda:0')
tensor(0.9846, device='cuda:0')
tensor(0.8523, device='cuda:0')
tensor(0.8597, device='cuda:0')


  3%|▎         | 154/4999 [14:51:03<493:15:50, 366.51s/it]

Checkpoint 154 saved !
------- 1st valloss=0.8033

tensor(0.6172, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.6953, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.6119, device='cuda:0', grad_fn=<DivBackward0>)
tensor(1., device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.6712, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.7745, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.7028, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.7448, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.6602, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.6667, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.7119, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.7750, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.7498, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.6555, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.5828, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.6030, device='cuda:0', grad_fn=<DivBackward0>)
tensor(1., device='cuda:0', grad_fn=<DivBackward0>)
tenso

In [ ]:
print(image_1_resize.shape)

In [ ]:
show_image_slice(image_1)
show_image_slice(label_1)
show_image_slice(bbox_bv_label)
show_image_slice(out_coarse)
show_image_slice(bbox_bv)
show_image_slice(bbox_image)

In [ ]:
b = loadbvcenter(image_1.squeeze(0).cpu().detach().numpy())
x1 = int(b[0])
y1 = int(b[1])
z1 = int(b[2])
show_image_slice(bv_label[x1-64:x1+64, y1-64:y1+64, z1-64:z1+64])
c = find_bv_centroid(bv_label)
x2 = int(c[0])
y2 = int(c[1])
z2 = int(c[2])
show_image_slice(bv_label[x2-64:x2+64, y2-64:y2+64, z2-64:z2+64])

In [ ]:
print(bv_coarse.shape)
print("bbox_bv_label", bbox_bv_label.shape)
print("bbox_bv", bbox_bv.shape)
print("bbox_image", bbox_image.shape)

In [ ]:
deeplab.eval()

with torch.no_grad():
    
    bgloss = 0
    bdloss = 0
    bvloss = 0
    
    for v, vbatch in tqdm(enumerate(validation_loader)):
        # move data to device, convert dtype to desirable dtype
        image_1 = vbatch['image1_data'].to(device=device, dtype=dtype)
        label_1 = vbatch['image1_label'].to(device=device, dtype=dtype)

        output = deeplab(image_1)
        # do the inference
        output_numpy = output.cpu().numpy()
        
        
        #out_1 = torch.round(output)
        out_1 = torch.from_numpy((output_numpy == output_numpy.max(axis=1)[:, None]).astype(int)).to(device=device, dtype=dtype)
        loss_1 = dice_loss_3(out_1, label_1)

        bg, bd, bv = dice_loss_3_debug(out_1, label_1)
        # calculate loss
        print(bg.item(), bd.item(), bv.item(), loss_1.item())
        bgloss += bg.item()
        bdloss += bd.item()
        bvloss += bv.item()
        
        if bv.item() >= 0.2 or bd.item() >= 0.1:
            show_image_slice(image_1)
            show_image_slice(label_1)
            show_image_slice(output)

    outstr = '------- background loss = {0:.4f}, body loss = {1:.4f}, bv loss = {2:.4f}'\
        .format(bgloss/(v+1), bdloss/(v+1), bvloss/(v+1)) + '\n'
    print(outstr)